In [2]:
import json

file = open("data_processing/paper_splits.json", "r")
paper_splits = json.loads(file.read())
print(len(paper_splits))

100


In [3]:
from sentence_transformers import SentenceTransformer
# import torch

# model = SentenceTransformer(
#     "sentence-transformers/distilbert-base-nli-mean-tokens",
#     device="cuda"
#     if torch.cuda.is_available()
#     else "mps"
#     if torch.backends.mps.is_available()
#     else "cpu",
# )

model = SentenceTransformer('sentence-transformers/allenai-specter').cuda() # this sentence transformer is the best so far and used in search engine

# Use chatgpt as a reference summarizer

In [13]:
# import openai
# import os
# from tqdm import tqdm

# openai.api_key = os.getenv("OPENAI_API_KEY")

# for paper in tqdm(paper_splits):
#     prompt = f"summarize the following document with max 100 words: {paper['body_split']}"
#     openAIresponse = openai.ChatCompletion.create(
#     model="gpt-3.5-turbo",
#     messages=[
#         {"role": "user", "content": prompt},
#     ],
#     max_tokens=128,
#     temperature=0.8,
#     )
#     paper["reference_summary"] = openAIresponse["choices"][0]["message"]["content"]

100%|██████████| 100/100 [08:06<00:00,  4.87s/it]


In [18]:
# outfile = open("paper_splits_output.json", "w")
# outfile.write(json.dumps(paper_splits))
# outfile.close()

In [4]:
paper_splits_output = json.loads(open(file="paper_splits_output.json").read())

# Compare current summary with another summarizer
The current summarizer is bart pretrained on news, now we want to compare it with another trained on another dataset

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
INSTRUCTION = "summarize the following article: "
tokenizer2 = AutoTokenizer.from_pretrained("haining/scientific_abstract_simplification")
model2 = AutoModelForSeq2SeqLM.from_pretrained("haining/scientific_abstract_simplification").cuda()

for paper in paper_splits_output:
    encoding = tokenizer2(paper["body_split"], 
                        max_length=512,
                        padding='max_length', 
                        truncation=True, 
                        return_tensors='pt')

    decoded_ids = model2.generate(input_ids=encoding['input_ids'],
                                attention_mask=encoding['attention_mask'], 
                                max_length=128, 
                                top_p=.9, 
                                do_sample=True)
    
    paper["summary_model2"] = decoded_ids

In [5]:
from tqdm import tqdm

for paper in tqdm(paper_splits_output):
    paper["reference_encoding"] = model.encode(paper["reference_summary"])
    paper["summary_encoding"] = model.encode(paper["summary"])

100%|██████████| 100/100 [00:18<00:00,  5.31it/s]


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
import pandas as pd

for split in paper_splits_output:
    split["cosine_similarity"] = cosine_similarity([split["reference_encoding"]],[split["summary_encoding"]])[0][0]
    # split["euclidean_distance"] = euclidean_distances([split["reference_encoding"]],[split["summary_encoding"]])[0][0]
    print(split["cosine_similarity"])
    # if(split["cosine_similarity"]<0):
    #     print(split["title"])

# paper_splits[0].keys()
df = pd.DataFrame.from_records(data=paper_splits_output)
    

In [8]:
import numpy as np

print(np.average(df["cosine_similarity"].tolist()))

0.8424217861890793
